# Init

In [1]:
# from skfem import *
import numpy as np
from utils import *
from skfem.helpers import d, dd, ddd, dot, ddot, grad, dddot, prod
from scipy.sparse.linalg import LinearOperator, minres
# from skfem.models.poisson import *
from skfem.assembly import BilinearForm, LinearForm
import datetime
import pandas as pd
import sys
import time
from tqdm import tqdm


tol = 1e-8
intorder = 3 # 6
solver_type = 'mgcg'
refine_time = 7
element_type = 'P1'
sigma = 5 
penalty = False
epsilon = 1e-6
ep = epsilon
example = 'ex3'
save_path = 'log/' + example + '_' + element_type + '_' + ('pen' if penalty else 'nopen') + '_' +'{}'.format(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

# Save $u_{h0}$ with different $h$ 

# %%time
# for i in range(7, 10):
#     m = MeshTri()
#     base_order = i
#     base_path = 'solutions/uh0_{}.npy'.format(base_order)
#     m.refine(base_order)

#     if penalty:
#         uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=6, tol=1e-8, epsilon=1e-6)
#     else:
#         uh0, basis = solve_problem1(m, element_type, solver_type, intorder=6, tol=1e-8, epsilon=1e-6)

#     np.save(base_path, uh0)
#     print('{} th saved'.format(i))

## Reading previous solution

In [2]:
test_order = 3
test_path = 'solutions/uh0_{}.npy'.format(test_order)
coarse_m = MeshTri()
coarse_m.refine(test_order)
test_basis, test_fbasis = solve_problem2(coarse_m, element_type, intorder=3, basis_only=True)
test_uh0 = np.load(test_path)

base_order = 5
base_path = 'solutions/uh0_{}.npy'.format(base_order)
fine_m = MeshTri()
fine_m.refine(base_order)
base_basis, base_fbasis = solve_problem2(fine_m, element_type, intorder=3, basis_only=True)
base_uh0 = np.load(base_path)

coordinates = base_basis['u'].global_coordinates().value

## $L2$ error

In [7]:
base_test_basis = np.zeros_like(base_basis['u'].interpolate(base_uh0).value)

# from numba import jit
# # @jit
# def test():
#     for i in tqdm(range(base_test_basis.shape[0])):
#         for j in range(base_test_basis.shape[1]):
#             base_test_basis[i][j] = test_basis['u'].interpolator(test_uh0)(np.array([[coordinates[0][i][j]], [coordinates[1][i][j]]]))
#     return base_test_basis

for i in tqdm(range(base_test_basis.shape[0])):
    for j in range(base_test_basis.shape[1]):
        base_test_basis[i][j] = test_basis['u'].interpolator(test_uh0)(np.array([[coordinates[0][i][j]], [coordinates[1][i][j]]]))

np.sqrt(np.sum(base_basis['u'].dx * (base_basis['u'].interpolate(base_uh0).value - base_test_basis)**2))

100%|██████████████████████████████████████████████████████████████████████████████| 2048/2048 [00:22<00:00, 91.63it/s]


0.025175332152123173

## Parallelize

In [10]:
base_test_basis = np.zeros_like(base_basis['u'].interpolate(base_uh0).value)

In [28]:
import concurrent.futures

def interpolator_parallel(j):
    for i in range(base_test_basis.shape[0]):
        base_test_basis[i][j] = test_basis['u'].interpolator(test_uh0)(np.array([[coordinates[0][i][j]], [coordinates[1][i][j]]]))
    print(j, 'done')

with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    list(executor.map(interpolator_parallel, range(base_test_basis.shape[1])))

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [26]:
base_test_basis

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

## $H_{1}$ Error

In [ ]:
dbasis = InteriorBasis(coarse_m, ElementTriDG(ElementTriP1()), intorder=3)
dx = project(test_uh0, basis_from=test_basis['u'], basis_to=dbasis, diff=0)
dy = project(test_uh0, basis_from=test_basis['u'], basis_to=dbasis, diff=1)


dux = np.zeros_like(base_basis['u'].interpolate(base_uh0).value)

for i in tqdm(range(base_test_basis.shape[0])):
    for j in range(base_test_basis.shape[1]):
        dux[i][j] = dbasis.interpolator(dx)(np.array([[coordinates[0][i][j]], [coordinates[1][i][j]]]))
        
duy = np.zeros_like(base_basis['u'].interpolate(base_uh0).value)

for i in tqdm(range(base_test_basis.shape[0])):
    for j in range(base_test_basis.shape[1]):
        duy[i][j] = dbasis.interpolator(dy)(np.array([[coordinates[0][i][j]], [coordinates[1][i][j]]]))

        
def get_DuError_N(basis, u):
    duh = basis.interpolate(u).grad
    x = basis.global_coordinates().value
    dx = basis.dx
    return np.sqrt(np.sum(((duh[0] - dux)**2 + (duh[1] - duy)**2) * dx))

get_DuError_N(base_basis['u'], base_uh0)

## $H_2$ Error 

In [ ]:
ddbasis = InteriorBasis(coarse_m, ElementTriDG(ElementTriP0()), intorder=3)
dxx = project(dx, basis_from=dbasis, basis_to=ddbasis, diff=0)
dxy = project(dx, basis_from=dbasis, basis_to=ddbasis, diff=1)
dyx = dxy
# dyx = project(dy, basis_from=dbasis, basis_to=ddbasis, diff=0)
dyy = project(dy, basis_from=dbasis, basis_to=ddbasis, diff=1)

duxx = np.zeros_like(base_basis['u'].interpolate(base_uh0).value)

for i in tqdm(range(base_test_basis.shape[0])):
    for j in range(base_test_basis.shape[1]):
        duxx[i][j] = ddbasis.interpolator(dxx)(np.array([[coordinates[0][i][j]], [coordinates[1][i][j]]]))
        
duxy = np.zeros_like(base_basis['u'].interpolate(base_uh0).value)

for i in tqdm(range(base_test_basis.shape[0])):
    for j in range(base_test_basis.shape[1]):
        duxy[i][j] = ddbasis.interpolator(dxy)(np.array([[coordinates[0][i][j]], [coordinates[1][i][j]]]))
        
duyx = duxy

# duyx = np.zeros_like(base_basis['u'].interpolate(base_uh0).value)

# for i in tqdm(range(base_test_basis.shape[0])):
#     for j in range(base_test_basis.shape[1]):
#         duyx[i][j] = ddbasis.interpolator(dyx)(np.array([[coordinates[0][i][j]], [coordinates[1][i][j]]]))
        
duyy = np.zeros_like(base_basis['u'].interpolate(base_uh0).value)

for i in tqdm(range(base_test_basis.shape[0])):
    for j in range(base_test_basis.shape[1]):
        duyy[i][j] = ddbasis.interpolator(dyy)(np.array([[coordinates[0][i][j]], [coordinates[1][i][j]]]))

        
def get_D2uError_N(basis, u):
    dduh = basis.interpolate(u).hess
    x = basis.global_coordinates().value
    dx = basis.dx
    return np.sqrt(np.sum(((dduh[0][0] - duxx)**2 + (dduh[0][1] - duxy)**2 +
                (dduh[1][1] - duyy)**2 + (dduh[1][0] - duyx)**2) * dx))

get_D2uError_N(base_basis['u'], base_uh0)

# Solving

In [ ]:
# solving 

print('=======Arguments=======')
print('penalty:\t{}'.format(penalty))
print('element_type:\t{}'.format(element_type))
print('solver_type:\t{}'.format(solver_type))
print('tol:\t{}'.format(tol))
print('intorder:\t{}'.format(intorder))
print('refine_time:\t{}'.format(refine_time))
print('epsilon:\t{}'.format(epsilon))
print('sigma:\t{}'.format(sigma))
print('=======Results=======')

time_start = time.time()

ep = epsilon
df_list = []
L2_list = []
Du_list = []
D2u_list = []
h_list = []
epu_list = []
m = MeshTri()

for i in range(1, refine_time+1):

    m.refine()

    if penalty:
        uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=intorder, tol=1e-8, epsilon=1e-6)
    else:
        uh0, basis = solve_problem1(m, element_type, solver_type, intorder=intorder, tol=1e-8, epsilon=1e-6)

    U = basis['u'].interpolate(uh0).value

    # compute errors

    L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
    Du = get_DuError(basis['u'], uh0)
    H1u = Du + L2u
    
    if penalty:
        D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
    else:
        D2u = get_D2uError(basis['u'], uh0)
        
    epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
    h_list.append(m.param())
    Du_list.append(Du)
    L2_list.append(L2u)
    D2u_list.append(D2u)
    epu_list.append(epu)

hs = np.array(h_list)
L2s = np.array(L2_list)
Dus = np.array(Du_list)
D2us = np.array(D2u_list)
epus = np.array(epu_list)
H1s = L2s + Dus
H2s = H1s + D2us

# store data
data = np.array([L2s, H1s, H2s, epus])
df = pd.DataFrame(data.T, columns=['L2', 'H1', 'H2', 'Energy'])
df_list.append(df)

show_result(L2s, H1s, H2s, epus)

# df.to_csv(save_path+'.csv')
time_end = time.time()
print('Total Time Cost {:.2f} s'.format(time_end-time_start))